## Spatial Joins

A Spatial join is a GIS operation that affixes data from one feature layer’s attribute table to another from a spatial perspective. Spatial joins begin by selecting a target feature and comparing it spatially to other feature layers. The target features will inherit the attributes from the other features if and only if the two features share the same spatial reference. Spatial join is a way that data from one feature class can be easily added to another feature class. 

Spatial joins use [binary predictions](https://shapely.readthedocs.io/en/latest/manual.html#predicates-and-relationships) such as contains and intersects to combine two Geodataframes based on the spatial relationship between the geometries.

### Types of spatial joins

### **Note** that *left* refers to the dataframe mentioned first (on the left) and *right* refers to the dataframe mentioned second (or on the right as you are reading the dataframes)

#### Left outer join

In a LEFT OUTER JOIN, *All* rows from the left are kept and duplicated if necessary to represent multiple hits between the two dataframes. Right rows are retained if they *intersect* a left row and are lost if they do not. A left outer join implies an interest in retaining the geometries of the left. 

![Left join ven diagram](https://www.codeproject.com/KB/database/Visual_SQL_Joins/LEFT_JOIN.png)

This is equivalent to the PostGIS query:
```
SELECT pts.geom, pts.id as ptid, polys.id as polyid  
FROM pts
LEFT OUTER JOIN polys
ON ST_Intersects(pts.geom, polys.geom);

                    geom                    | ptid | polyid 
--------------------------------------------+------+--------
 010100000040A9FBF2D88AD03F349CD47D796CE9BF |    4 |     10
 010100000048EABE3CB622D8BFA8FBF2D88AA0E9BF |    3 |     10
 010100000048EABE3CB622D8BFA8FBF2D88AA0E9BF |    3 |     20
 0101000000F0D88AA0E1A4EEBF7052F7E5B115E9BF |    2 |     20
 0101000000818693BA2F8FF7BF4ADD97C75604E9BF |    1 |       
(5 rows)
```


#### Right outer join

In a RIGHT OUTER JOIN, *All* rows from the right are kept and duplicated if necessary to represent multiple hits between the two dataframes. Left rows are retained if they *intersect* a right row and are lost if they do not. A right outer join implies an interest in retaining the geometries of the right. 

![right join ven diagram](https://www.codeproject.com/KB/database/Visual_SQL_Joins/RIGHT_JOIN.png)

This is equivalent to the PostGIS query:
```
SELECT polys.geom, pts.id as ptid, polys.id as polyid  
FROM pts
RIGHT OUTER JOIN polys
ON ST_Intersects(pts.geom, polys.geom);

  geom    | ptid | polyid 
----------+------+--------
 01...9BF |    4 |     10
 01...9BF |    3 |     10
 02...7BF |    3 |     20
 02...7BF |    2 |     20
 00...5BF |      |     30
(5 rows)
```

#### Inner join

In an INNER JOIN, rows from the right and left are only kept where their binary predicate is `True`. We duplicate them if necessary, to represent multiple hits between the two dataframes. Attributes of the right and left are only retained if they *intersect* and are lost if they do not. An inner join implies an interest in retaining the geometries that overlap.

![inner join ven diagram](https://www.codeproject.com/KB/database/Visual_SQL_Joins/INNER_JOIN.png)

This is equivalent to the PostGIS query:
```
SELECT pts.geom, pts.id as ptid, polys.id as polyid  
FROM pts
INNER JOIN polys
ON ST_Intersects(pts.geom, polys.geom);

                    geom                    | ptid | polyid 
--------------------------------------------+------+--------
 010100000040A9FBF2D88AD03F349CD47D796CE9BF |    4 |     10
 010100000048EABE3CB622D8BFA8FBF2D88AA0E9BF |    3 |     10
 010100000048EABE3CB622D8BFA8FBF2D88AA0E9BF |    3 |     20
 0101000000F0D88AA0E1A4EEBF7052F7E5B115E9BF |    2 |     20
(4 rows) 
```

Now let's load some data and see these joins in action. Once again, we will be using the NYC data

 * [NYC Borough Boundaries](https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile) - See it in action [here](https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm)
 * [NYC Wi-Fi Hotspot Locations](https://data.cityofnewyork.us/api/geospatial/a9we-mtpn?method=export&format=Shapefile) - See it in action [here](https://data.cityofnewyork.us/Social-Services/NYC-Wi-Fi-Hotspot-Locations/a9we-mtpn)


##### Local File Alternatives for fall-back downloads
 * [nyc_boroughs.zip](https://web.dsa.missouri.edu/static/data/geodata/nyc_boroughs.zip)
 * [nyc_wifi.zip](https://web.dsa.missouri.edu/static/data/geodata/nyc_wifi.zip)
 

In [ ]:
import os
import pandas as pd
import numpy as np
from shapely.geometry import Point
from random import randint, uniform
import matplotlib.pyplot as plt

import urllib.request
import shutil
from pathlib import Path
from zipfile import ZipFile

import geopandas as gpd

In [ ]:
# Designate the URL for a file
file_URL = 'https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile'

# Designate the local filename
local_file_name = 'nyc_boroughs.zip'

# Designate the local file name with a path to a temp directory.
     # Your Repo comes with this folder.  If not, use terminal and 
     # navigate to course folder > module3 and then : mkdir temp
file_Path = Path('../temp/')  
file_Path /= local_file_name

# Download the file from `url` and save it locally under `file_Path`:
with urllib.request.urlopen(file_URL) as response,  file_Path.open(mode='w+b') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip file
to_unzip = ZipFile('../temp/nyc_boroughs.zip', 'r')
unzipped = '../temp/nyc_boroughs_unzipped'
to_unzip.extractall(unzipped)
to_unzip.close()
for file in os.listdir(unzipped):
    if file.endswith(".shp"):
        shape_file =  unzipped + '/' + file

NYC = gpd.GeoDataFrame.from_file(shape_file)        

In [ ]:
# Designate the URL for a file
file_URL = 'https://data.cityofnewyork.us/api/geospatial/a9we-mtpn?method=export&format=Shapefile'

# Designate the local filename
local_file_name = 'nyc_wifi.zip'

# Designate the local file name with a path to a temp directory.
     # Your Repo comes with this folder.  If not, use terminal and 
     # navigate to course folder > module3 and then : mkdir temp
file_Path = Path('../temp/')  
file_Path /= local_file_name

# Download the file from `url` and save it locally under `file_Path`:
with urllib.request.urlopen(file_URL) as response,  file_Path.open(mode='w+b') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip file
to_unzip = ZipFile('../temp/nyc_wifi.zip', 'r')
unzipped = os.getcwd() + '/nyc_wifi_unzipped'
to_unzip.extractall(unzipped)
to_unzip.close()
for file in os.listdir(unzipped):
    if file.endswith(".shp"):
        shape_file =  unzipped + '/' + file

# Read data into GeoDataFrame and view data
NYC_wifi = gpd.GeoDataFrame.from_file(shape_file)

In [ ]:
NYC_wifi.plot(ax=NYC.plot(), color='red', markersize=1)    

Once again, we are also going to separate the boroughs to showcase the joins better.

In [ ]:
Staten_Island = NYC[NYC['boro_name'] == 'Staten Island']
Manhattan = NYC[NYC['boro_name'] == 'Manhattan']
Brooklyn = NYC[NYC['boro_name'] == 'Brooklyn']
Queens = NYC[NYC['boro_name'] == 'Queens']
Bronx = NYC[NYC['boro_name'] == 'Bronx']

## Joins

In [ ]:
from geopandas.tools import sjoin
join_left_df = sjoin(NYC_wifi, Queens, how="left")
join_left_df

*Note the NaN where the point was not in Queens*

In [ ]:
join_right_df = sjoin(NYC_wifi, Queens, how="right")
join_right_df

*Note that we only have the Queens information and it is repeated and added to each wi-fi point*

In [ ]:
join_inner_df = sjoin(NYC_wifi, Queens, how="inner")
join_inner_df
# Note the lack of NaNs; dropped anything that didn't intersect

As you can see, we got the same results for the right join as we did for the inner join. Why is this?

You are not limited to using the intersection binary predicate. Any of the Shapely geometry methods that return a Boolean can be used by specifying the op keyword.

In [ ]:
sjoin(NYC_wifi, Queens, how="inner", op='within')

## Conclusion
This lab covered:
* Spatial joins using the geopanda sjoin function 